In [1]:
import pandas as pd
import numpy as np
from cvxpy import *

In [2]:
def expected_return(df):
    # read monthly_prices.csv
    mp = df
    mr = pd.DataFrame()

    # compute monthly returns
    for s in mp.columns:
        date = mp.index[0]
        pr0 = mp[s][date] 
        for t in range(1,len(mp.index)):
            date = mp.index[t]
            pr1 = mp[s][date]
            ret = (pr1-pr0)/pr0
            mr.at[date, s] = ret
            pr0 = pr1

    # get symbol names
    symbols = mr.columns

    # convert monthly return data frame to a numpy matrix
    return_data = mr.to_numpy().T

    # compute mean return
    r = np.asarray(np.mean(return_data, axis=1))

    # covariance
    C = np.asmatrix(np.cov(return_data))

    # print out expected return and std deviation
    print ("----------------------")
    for j in range(len(symbols)):
        print ('%s: Exp ret = %f, Risk = %f' %(symbols[j],r[j], C[j,j]**0.5))


    # set up optimization model
    n = len(symbols)
    x = Variable(n)
    req_return = 0.01
    ret = r.T@x
    risk = quad_form(x, C)
    prob = Problem(Minimize(risk), 
                   [sum(x) == 1, ret >= req_return,
                    x >= 0])

    # solve problem and write solution
    try:
        prob.solve()
        print ("----------------------")
        print ("Optimal portfolio")
        print ("----------------------")
        for s in range(len(symbols)):
            print ('{}'.format(symbols[s]) + ' = ${:.2f}'.format(round(x.value[s] * 1000,2)))
        print ("----------------------")
        print ('Exp ret = %f' %(ret.value))
        print ('risk    = %f' %((risk.value)**0.5))
        print ("----------------------")
    except:
        print ('Error')

In [3]:
in_class_mp = pd.read_csv('monthly_prices.csv', index_col = 0)
expected_return(in_class_mp)

----------------------
MSFT: Exp ret = 0.024611, Risk = 0.058040
V: Exp ret = 0.018237, Risk = 0.042807
WMT: Exp ret = 0.009066, Risk = 0.044461
----------------------
Optimal portfolio
----------------------
MSFT = $100.50
V = $414.33
WMT = $485.16
----------------------
Exp ret = 0.014428
risk    = 0.025495
----------------------


In [12]:
import yfinance as yf

data = yf.download("MSFT V WMT", start="2020-09-01", end="2022-08-24", interval="1mo")
test = data
data = data.dropna()
data = data['Adj Close']
data = data.reset_index()
data = data.drop(columns='Date')

expected_return(data)

[*********************100%***********************]  3 of 3 completed
----------------------
MSFT: Exp ret = 0.013214, Risk = 0.064394
V: Exp ret = 0.003721, Risk = 0.074425
WMT: Exp ret = 0.000418, Risk = 0.062550
----------------------
Optimal portfolio
----------------------
MSFT = $687.70
V = $236.74
WMT = $75.56
----------------------
Exp ret = 0.010000
risk    = 0.054507
----------------------
